Set up workspace variables.

In [1]:
library(AzureML)
workspace_id <- '1ca5ada1f2dc4a7a8a50c40eea394d1f'
authorization_token <- 'c24c13e913e84a959413295d09e1b361'
ws <- workspace(workspace_id, authorization_token)

Get the Prestige dataset.

In [8]:
Prestige <- download.datasets(ws, name = "Prestige.csv")

User Linear Regression.

In [3]:
model <- lm(prestige ~ education + log2(income) + type, Prestige)

Check some predictions.

In [19]:
test.data = list(education=12.5, income=10000, type='prof')
predict(model, test.data)

1 
63.19827

Build and publish the web service.

In [9]:
services(ws)

Id,Name,Description,CreationTime,WorkspaceId,DefaultEndpointName,EndpointCount
c758dfe5ed004fe8a4ce715085257863,Iris Dataset Binary Classification [Predictive Exp.],No description provided for this web service.,2017-08-26T14:03:37.34Z,1ca5ada1f2dc4a7a8a50c40eea394d1f,default,1


In [16]:
wrapper_function <- function(education, income, type) {
    predict(model, list(education=education, income=income, type=type))
}

prestigeWebService <- publishWebService(ws = ws, 
                        fun = wrapper_function, 
                        name = "Prestige Prediction", 
                        inputSchema = list(
                            education = "numeric", 
                            income = "numeric",
                            type = "character"
                          ), 
                        outputSchema = list(
                            res = "numeric"
                          ))

Update if needed.

In [18]:
wrapper_function <- function(education, income, type) {
    predict(model, list(education=education, income=income, type=type))
}

prestigeWebService <- updateWebService(ws = ws, 
                        fun = wrapper_function, 
                        name = "Prestige Prediction", 
                        inputSchema = list(
                            education = "numeric", 
                            income = "numeric",
                            type = "character"
                          ), 
                        outputSchema = list(
                            res = "numeric"
                          ),
                        serviceId = prestigeWebService$WebServiceId   # Required to update
                        )

Use the webservice to predict new values.

In [20]:
consume(prestigeWebService, test.data)

res
63.19827
